In [ ]:
from script.requirements import *
from datetime import date
from pycelonis.pql import PQL, PQLColumn, PQLFilter, OrderByColumn
import pm4py

In [ ]:
data_model = data_pool.get_data_models().find("xxx")

In [ ]:
query = PQL(distinct=False, limit=None, offset=None)
query += PQLColumn(name="_CASE_KEY", query=""" "_CEL_MERGED_ACTIVITIES"."CASE_ID" """)
query += PQLColumn(name="WERKS", query=""" "_CEL_MERGED_ACTIVITIES"."WERKS" """)
query += PQLColumn(name="EBELN", query=""" "_CEL_MERGED_ACTIVITIES"."EBELN" """)
query += PQLColumn(name="EBELP", query=""" "_CEL_MERGED_ACTIVITIES"."EBELP" """)
query += PQLColumn(name="ACTIVITY_EN", query=""" "_CEL_MERGED_ACTIVITIES"."ACTIVITY" """)
query += PQLColumn(name="EVENTTIME", query=""" "_CEL_MERGED_ACTIVITIES"."TIMESTAMP" """)
query += PQLColumn(name="_SORTING", query=""" "_CEL_MERGED_ACTIVITIES"."_SORTING" """)
query += PQLColumn(name="USER_NAME", query=""" "_CEL_MERGED_ACTIVITIES"."USER_NAME" """)
query += PQLColumn(name="RKDST", query=""" "MARA"."RKDST" """)
query += PQLColumn(name="NAME1", query=""" "MARA"."NAME1" """)
query += PQLColumn(name="ELIKZ", query=""" "EKPO"."ELIKZ" """)
query += PQLColumn(name="MATNR", query=""" "EKPO"."MATNR" """)
query += PQLColumn(name="MATKL", query=""" "EKPO"."MATKL" """)
query += PQLColumn(name="MENGE", query=""" "EKPO"."MENGE" """)
query += PQLColumn(name="DELGE", query=""" "EKPO"."DELGE" """)
query += PQLColumn(name="NETPR", query=""" "EKPO"."NETPR" """)

query += PQLColumn(name="MAVPR", query=""" CASE WHEN
"MARA"."RKDST" = 'x' AND
PROCESS ON "_CEL_MERGED_ACTIVITIES"."ACTIVITY" EQUALS
'Create Purchase Requisition Item' TO 'Purchase Requisition Item: Rejected' TO 'Create Purchase Order Item' THEN 1

WHEN
"MARA"."RKDST" = 'x' AND
"EBAN"."EBELN" IS NULL THEN 1

WHEN
PROCESS ON "_CEL_MERGED_ACTIVITIES"."ACTIVITY" EQUALS
'Create Purchase Order Item' TO 'Purchase Order Item: Rejected' TO 'Purchase Order Item: Sent to Vendor' THEN 1

WHEN
PROCESS ON "_CEL_MERGED_ACTIVITIES"."ACTIVITY" EQUALS
'Purchase Order Item: Sent to Vendor' TO ANY TO 'Change Quantity' THEN 1

WHEN
PROCESS ON "_CEL_MERGED_ACTIVITIES"."ACTIVITY" EQUALS
'Purchase Order Item: Sent to Vendor' TO ANY TO 'Change Price' THEN 1

ELSE NULL END """)

query += PQLColumn(name="THWMS", query=""" CASE WHEN 
"RSEG"."EBELN" IS NOT NULL AND
"RSEG"."MENGE" = "EKPO"."MENGE" AND
"RSEG"."MENGE" = "EKPO"."DELGE" AND
"EKPO"."MENGE" = "EKPO"."DELGE" AND 
"RSEG"."NETPR"="EKPO"."NETPR" THEN NULL
WHEN
"RSEG"."EBELN" IS NULL
THEN NULL
ELSE 1 END """)

query += PQLColumn(name="LATES", query=""" CASE WHEN
PROCESS ON "_CEL_MERGED_ACTIVITIES"."ACTIVITY" EQUALS 'Late Shipment' THEN 1 ELSE 0 END """)

query += PQLColumn(name="LATED", query=""" CASE WHEN
PROCESS ON "_CEL_MERGED_ACTIVITIES"."ACTIVITY" EQUALS 'Late Delivery' THEN 1 ELSE 0 END """)

query += PQLColumn(name="LATEP", query=""" CASE WHEN 
"RBKP"."BPDAT">"RBKP"."BUDAT" THEN
1

WHEN
"RBKP"."BPDAT" IS NULL AND
"RBKP"."BUDAT" < TODAY() THEN
1

ELSE NULL END """)

query += PQLFilter(query=""" FILTER ( "EKPO"."ELIKZ" ='x' and "RBKP"."EREKZ" ='x') """)
#query += PQLFilter(query=""" FILTER ( "EKPO"."ELIKZ" IS NULL ) """)

result_df = data_model.export_data_frame(query)
result_df.to_csv('df_completed_order.csv', index = False)